#### 1. 加载模型

In [1]:
from langchain_community.llms import Tongyi
from langchain_community.embeddings import DashScopeEmbeddings
# 创建一个Tongyi类的实例，使用模型 'qwen-max'，你也可以将使用其他模型，比如将 qwen-max 替换为 qwen2-72b-instruct
llm = Tongyi(model_name='qwen-plus')

# 使用实例的方法 invoke 调用模型服务，传入提示词
# result1 = llm.invoke("2024年的金球奖谁拿了？") # 原始模型无法检索到实时信息
# print(result1)
# result2 = llm.invoke("曼城在哪里？") # 原始模型对主语有一定的误解，如果有上下文的基础，应该将这里的“曼城”理解为曼彻斯特城足球俱乐部。
# print(result2)

# 加载 embedding创建模型
embeddings = DashScopeEmbeddings(model="text-embedding-v3")

#### 2. 加载数据并切片

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownTextSplitter
import os

# 初始化文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # 每个切片的字符数
    chunk_overlap=50,  # 切片之间的重叠字符数
    separators=["\n\n", "\n", "。", " ","，"],  # 切分依据
)
#text_splitter = MarkdownTextSplitter(chunk_size=500, chunk_overlap=50)

# 加载 Markdown 文本
# with open('deepseekv3.md', "r", encoding="utf-8") as file:
#     markdown_content = file.read()
#     print(f"总字符数：{len(markdown_content)}")

# 存储所有切片的列表
all_chunks = []

# 遍历 ./news 目录下的所有文件
for filename in os.listdir('./news'):
    if filename.endswith('.md'):
        file_path = os.path.join('./news', filename) 

        with open(file_path, "r", encoding="utf-8") as file:
            markdown_content = file.read()
        
        # 输出文件字符数
        char_count = len(markdown_content)
        print(f"文件: {filename}，字符数: {char_count}")

        # 将 Markdown 文本切片
        chunks = text_splitter.split_text(markdown_content)
        all_chunks.extend(chunks)

print(f"生成文本块数量: {len(all_chunks)}") 

文件: 体育-23年欧冠.md，字符数: 6679
文件: 体育-24年欧冠.md，字符数: 1952
文件: 体育-东77.md，字符数: 405
文件: 体育-内马尔.md，字符数: 3390
文件: 体育-巴特勒.md，字符数: 1185
文件: 体育-欧冠综合20250130.md，字符数: 562
文件: 体育-欧冠综合20250131.md，字符数: 1222
文件: 体育-欧冠附加赛赛制.md，字符数: 2611
文件: 体育-福克斯.md，字符数: 3047
文件: 体育-篮球欧冠.md，字符数: 954
文件: 体育-英超20250131.md，字符数: 3472
文件: 体育-金球奖.md，字符数: 1146
文件: 健康-核桃.md，字符数: 1510
文件: 军事-东部战区.md，字符数: 1238
文件: 民生-春晚.md，字符数: 5288
文件: 民生-电影20250130.md，字符数: 273
文件: 民生-电影简介.md，字符数: 1441
文件: 科技-deepseekr1.md，字符数: 1823
文件: 财经-2024GDP.md，字符数: 5831
生成文本块数量: 115


#### 3. 创建文本索引数据库并保存（第一次使用）

In [3]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance
from langchain_qdrant import QdrantVectorStore

print("加载Qdrant向量数据库中")
# 客户端模式
# client = QdrantClient(":memory:")

# 本地模式
client = QdrantClient(path="qdrant_data", port=6333)
# if not exists then create collection
if not client.collection_exists("rag_collection"):
    # create collection
    client.create_collection(
        "rag_collection",
        vectors_config=VectorParams(
            size=len(embeddings.embed_query("hello world")), # size:1536
            distance=Distance.COSINE
        )
    )
vector_store = QdrantVectorStore(client=client, collection_name="rag_collection", embedding=embeddings) # 初始化向量数据库
print("加载Qdrant向量数据库完成")
vector_store.add_texts(all_chunks) # 将切分好的文本嵌入到向量数据库中 （使用本地模式不要运行）
print("嵌入生成完成，向量数据库存储完成.") 
print("索引过程完成.") 

加载Qdrant向量数据库中
加载Qdrant向量数据库完成
嵌入生成完成，向量数据库存储完成.
索引过程完成.


##### 索引数据检索测试(向量相似度)

In [4]:
queries = ["欧冠直通16强淘汰赛的球队有哪些？", "曼城在附加赛的对手是谁？"]

vector_retriever = vector_store.as_retriever(search_kwargs={"k": 5})  # 只检索5条
# 批量查询，每个 query 都会返回 k 个最相关的文档
results = [vector_retriever.get_relevant_documents(query, k=5) for query in queries]

# 展示所有查询结果
for i, result in enumerate(results):
    print(f"🔍 查询: {queries[i]}")
    for i,doc in enumerate(result):
        print(f"检索到的段落{i+1} : {doc.page_content[:100]}")
    print("-" * 50)



C:\Users\ASUS\AppData\Local\Temp\ipykernel_55836\2952464770.py:5: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = [vector_retriever.get_relevant_documents(query, k=5) for query in queries]


🔍 查询: 欧冠直通16强淘汰赛的球队有哪些？
检索到的段落1 : #### 欧冠综合｜利物浦和巴萨直通16强
新华社 2025-01-30 11:12 北京
新华社柏林1月29日电（记者刘旸）本赛季欧冠联赛第一阶段29日结束全部8轮比赛，利物浦、巴萨、阿森纳等积分榜
检索到的段落2 : 意甲：8支球队参加欧战尚存7支，5支欧冠2支欧联1支欧协联。其中参加欧冠的5支球队中仅有国米以联赛第4名直接晋级淘汰赛，亚特兰大、AC米兰和尤文图斯都需要通过附加赛来争夺晋级资格，只有新手博洛尼亚以第
检索到的段落3 : 在欧冠淘汰赛附加赛中，常规赛排名最高的第9或第10位球队，将对阵排名最低的第23或24位球队；排名第11或第12位的球队，将对阵排名第21或第22位的球队；排名第13或第14位的球队，将对阵排名第19
检索到的段落4 : 面对36支球队采取联赛总排名的欧冠全新赛制，有的传统强队有点懵圈，曼城、皇马、拜仁都表现不佳。最后一轮联赛开始前，竟然仅有利物浦和巴萨锁定16强入场券。9支种子队除了卫冕冠军皇马之外，其余球队按照欧战
检索到的段落5 : 也就是说，曼城必对皇马和拜仁之一。尽管曼城本赛季整体表现不佳，但在冬季转会窗引进三名强援，分别是前锋马尔穆什以及两名中卫库萨诺夫和雷斯。曼城三名强援没有在欧冠常规赛注册，但注册以后有机会参加附加赛以及
--------------------------------------------------
🔍 查询: 曼城在附加赛的对手是谁？
检索到的段落1 : **曼城对决**
在抽签仪式上，首先抽出第23、第24名球队，然后依次抽出剩余非种子球队。随后，首先抽出第15、16名球队，然后依次抽出种子球队。附加赛没有回避规则，球队可以对阵同一足协球队以及联赛阶
检索到的段落2 : #### 欧冠附加赛抽签：曼城再战皇马，5年4次决战，拜仁大巴黎好签
2025-01-31 19:29:28　来源: 奥拜尔 
北京时间1月31日19时，2024-25赛季欧冠淘汰赛附加赛抽签在瑞士尼
检索到的段落3 : 也就是说，曼城必对皇马和拜仁之一。尽管曼城本赛季整体表现不佳，但在冬季转会窗引进三名强援，分别是前锋马尔穆什以及两名中卫库萨诺夫和雷斯。曼城三名强援没有在欧冠常规赛注册，但注册以后有机会参加附加赛以及
检索到的段落4 : AC米兰客场1:

#### 关键词索引


In [ ]:
# 方法一：该方法需要改写langchain的BaseRetriever类，比较麻烦

queries = ["欧冠直通16强淘汰赛的球队有哪些？","曼城在附加赛的对手是谁？"]
from rank_bm25 import BM25Okapi # 从 rank_bm25 库中导入 BM25Okapi 类，用于实现 BM25 算法的检索功能
import jieba # 导入 jieba 库，用于对中文文本进行分词处理
# 对所有文档进行中文分词 
tokenized_corpus = [list(jieba.cut(doc)) for doc in all_chunks]
# 使用分词后的文档集合实例化 BM25Okapi，对这些文档进行 BM25 检索的准备工作 
bm25 = BM25Okapi(tokenized_corpus)
# 定义需要返回的 top k 个文档
top_k = 5

for query in queries:
    # 对查询语句进行分词处理，将分词结果存储为列表 
    tokenized_query = list(jieba.cut(query))
    print(f"查询语句分词结果: {tokenized_query}")
    # 计算查询语句与每个文档的 BM25 得分，返回每个文档的相关性分数 
    bm25_scores = bm25.get_scores(tokenized_query)
    # 获取 BM25 检索得分最高的前 top_k 个文档的索引 
    bm25_top_k_indices = sorted(range(len(bm25_scores)), key=lambda i: bm25_scores[i], reverse=True)[:top_k]
    # 根据索引提取对应的文档内容 
    bm25_chunks = [all_chunks[i] for i in bm25_top_k_indices]
    # 输出检索结果
    print(f"查询：{query}")
    print(f"BM25 检索最相似的前 {top_k} 个文本块:")
    for rank, doc in enumerate(bm25_chunks):
        print(f"BM25 检索排名: {rank + 1}")
        # 如果文档是分词后的列表，可以用空格拼接恢复成字符串展示
        print("文档内容:")
        print(doc)
        print()

In [5]:
# 方法二：直接使用langchain提供的BM25Retriever，但要注意中文分词

from langchain.retrievers import BM25Retriever, EnsembleRetriever
import jieba # 导入 jieba 库，用于对中文文本进行分词处理
queries = ["欧冠直通16强淘汰赛的球队有哪些？","曼城在附加赛的对手是谁？"]

bm25_retriever = BM25Retriever.from_texts(all_chunks, preprocess_func=lambda text:" ".join(jieba.cut(text)))
bm25_retriever.k =  5  # Retrieve top 5 results

for query in queries:
    print(f"🔍查询：{query}")
    print(f"BM25 检索最相似的前 5 个文本块:")
    docs = bm25_retriever.invoke(query)
    for i, doc in enumerate(docs):
        print(f"检索到的段落{i+1} : {doc.page_content[:100]}")
        print()


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ASUS\AppData\Local\Temp\jieba.cache
Loading model cost 0.454 seconds.
Prefix dict has been built successfully.


🔍查询：欧冠直通16强淘汰赛的球队有哪些？
BM25 检索最相似的前 5 个文本块:
检索到的段落1 : #### 五大联赛欧战积分排名 ，英超无愧第一联赛，意甲西甲相差无几
新欧洲三大杯联赛阶段已经全部结束，每个杯赛各有8支球队直接晋级下阶段淘汰赛，16支球队将参加淘汰赛附加赛争夺另外8个晋级名额，还有

检索到的段落2 : 意甲：8支球队参加欧战尚存7支，5支欧冠2支欧联1支欧协联。其中参加欧冠的5支球队中仅有国米以联赛第4名直接晋级淘汰赛，亚特兰大、AC米兰和尤文图斯都需要通过附加赛来争夺晋级资格，只有新手博洛尼亚以第

检索到的段落3 : 德甲：7支球队参加欧战，4支欧冠2支欧联杯1支欧协联。其中参加欧冠的勒沃库森直接晋级淘汰赛，多特蒙德和拜仁将参加淘汰赛附加赛，斯图加特和莱比锡被直接淘汰。参加欧联杯的法兰克福直接晋级淘汰赛，霍芬海姆被

检索到的段落4 : 面对36支球队采取联赛总排名的欧冠全新赛制，有的传统强队有点懵圈，曼城、皇马、拜仁都表现不佳。最后一轮联赛开始前，竟然仅有利物浦和巴萨锁定16强入场券。9支种子队除了卫冕冠军皇马之外，其余球队按照欧战

检索到的段落5 : #### 欧冠新赛制格外刺激，曼城将在附加赛对阵皇马或拜仁 
2025-01-30 11:30 发布于：广东省
史上第一次，欧冠正赛在北京时间1月30日凌晨4时同时开始了多达18场比赛，堪比足球春晚，

🔍查询：曼城在附加赛的对手是谁？
BM25 检索最相似的前 5 个文本块:
检索到的段落1 : #### 巴尔扎雷蒂：米兰总是先丢球没进步；尤文图斯前场创造力不高
小科爱足球2025-01-30 14:56
费德里科-巴尔扎雷蒂，这位前乌迪内斯高层、现负责罗马球探和租借事务的主管，在接受亚马逊P

检索到的段落2 : #### 欧冠附加赛抽签：曼城再战皇马，5年4次决战，拜仁大巴黎好签
2025-01-31 19:29:28　来源: 奥拜尔 
北京时间1月31日19时，2024-25赛季欧冠淘汰赛附加赛抽签在瑞士尼

检索到的段落3 : #### 欧冠新赛制格外刺激，曼城将在附加赛对阵皇马或拜仁 
2025-01-30 11:30 发布于：广东省
史上第一次，欧冠正赛在北京时间1月30日凌晨4时同时开始了多达18场比赛，堪比足球春晚，

检索到的段落4 : 更难能可贵的是，眼下瓜迪奥

##### 混合检索

In [6]:
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, vector_retriever], weights=[0.4, 0.6])

processed_query = " ".join(jieba.cut(query))
ensemble_retriever.get_relevant_documents(processed_query)

[Document(metadata={}, page_content='#### 欧冠附加赛抽签：曼城再战皇马，5年4次决战，拜仁大巴黎好签\n2025-01-31 19:29:28\u3000来源: 奥拜尔 \n北京时间1月31日19时，2024-25赛季欧冠淘汰赛附加赛抽签在瑞士尼翁欧足联总部进行。在附加赛里，曼城将对阵皇马，这将是欧冠决赛的翻版。\n**抽签规则**\n欧足联本赛季对欧冠、欧联杯赛制进行改革，联赛阶段前8名直接晋级16强，排名第9到第24位的球队通过两回合附加赛拿到另外8张门票。\n按照规则，排名第9到第16位的球队成为附加赛种子队，根据联赛阶段排名决定潜在对手，排名第9、第10的球队将对阵第23、24的球队，以此类推。\n同时，在八分之一决赛抽签中，联赛阶段排名第1、2位的球队，对手将是附加赛第15、16、17、18名球队的球队，以此类推。\n在欧冠联赛阶段，亚特兰大、多特蒙德、皇马、拜仁、AC米兰、埃因霍温、巴黎圣日耳曼、本菲卡排名第9到第16位，摩纳哥、布雷斯特、费耶诺德、尤文、凯尔特人、曼城、葡萄牙体育、布鲁日排名第17到第24位。按照规则，凯尔特人、曼城将分别对阵皇马、拜仁。\n**曼城对决**'),
 Document(metadata={}, page_content='#### 欧冠新赛制格外刺激，曼城将在附加赛对阵皇马或拜仁 \n2025-01-30 11:30 发布于：广东省\n史上第一次，欧冠正赛在北京时间1月30日凌晨4时同时开始了多达18场比赛，堪比足球春晚，36支球队排定常规赛名次。欧冠常规赛排名很重要，曼城虽然进入附加赛，但必将与皇家马德里或拜仁慕尼黑之一争夺16强入场券，这就意味着这三支前欧冠冠军必然有一支无缘16强。\n本赛季欧冠首次实行新赛制，2022-2023赛季冠军曼城却成为表现最差的传统强队之一。曼城仅以第22名的身份进入附加赛，潜在对手是过去三个赛季两次夺得欧冠冠军的皇马，另一个潜在对手是2019-2020赛季冠军拜仁。\n很多美国资本已经进军欧洲足坛，欧足联在确立欧冠新赛制时，在一定程度上参考了NBA季后赛赛制，欧冠常规赛排名对淘汰赛落位很重要。\n欧冠淘汰赛附加赛抽签虽然种子队必对非种子队，但并不是完全随机抽取，而是参考排名加以限制，排名较高的球队原则上抽到实力较弱的球队，可惜常规赛排名并不能与

#### 重排序

In [7]:
# 方法一: LongContextReorder
from langchain_community.document_transformers import LongContextReorder


query = "欧冠直通16强淘汰赛的球队有哪些？"
print(f"问题：{query}")

docs = ensemble_retriever.get_relevant_documents(query) 
print("混合检索结果：")
for i,doc in enumerate(docs):
    print(f"检索到的段落{i+1} :{doc.page_content[:100]}")

print("-------------------")
print("重排序后结果：")
# Reorder the documents:
# Less relevant document will be at the middle of the list and more
# relevant elements at beginning / end.
reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs)
 
# Confirm that the 4 relevant documents are at beginning and end.
print("混合检索结果：")
for i,doc in enumerate(reordered_docs):
    print(f"检索到的段落{i+1} :{doc.page_content[:100]}")


问题：欧冠直通16强淘汰赛的球队有哪些？
混合检索结果：
检索到的段落1 :意甲：8支球队参加欧战尚存7支，5支欧冠2支欧联1支欧协联。其中参加欧冠的5支球队中仅有国米以联赛第4名直接晋级淘汰赛，亚特兰大、AC米兰和尤文图斯都需要通过附加赛来争夺晋级资格，只有新手博洛尼亚以第
检索到的段落2 :面对36支球队采取联赛总排名的欧冠全新赛制，有的传统强队有点懵圈，曼城、皇马、拜仁都表现不佳。最后一轮联赛开始前，竟然仅有利物浦和巴萨锁定16强入场券。9支种子队除了卫冕冠军皇马之外，其余球队按照欧战
检索到的段落3 :#### 欧冠综合｜利物浦和巴萨直通16强
新华社 2025-01-30 11:12 北京
新华社柏林1月29日电（记者刘旸）本赛季欧冠联赛第一阶段29日结束全部8轮比赛，利物浦、巴萨、阿森纳等积分榜
检索到的段落4 :在欧冠淘汰赛附加赛中，常规赛排名最高的第9或第10位球队，将对阵排名最低的第23或24位球队；排名第11或第12位的球队，将对阵排名第21或第22位的球队；排名第13或第14位的球队，将对阵排名第19
检索到的段落5 :也就是说，曼城必对皇马和拜仁之一。尽管曼城本赛季整体表现不佳，但在冬季转会窗引进三名强援，分别是前锋马尔穆什以及两名中卫库萨诺夫和雷斯。曼城三名强援没有在欧冠常规赛注册，但注册以后有机会参加附加赛以及
检索到的段落6 :#### 五大联赛欧战积分排名 ，英超无愧第一联赛，意甲西甲相差无几
新欧洲三大杯联赛阶段已经全部结束，每个杯赛各有8支球队直接晋级下阶段淘汰赛，16支球队将参加淘汰赛附加赛争夺另外8个晋级名额，还有
检索到的段落7 :德甲：7支球队参加欧战，4支欧冠2支欧联杯1支欧协联。其中参加欧冠的勒沃库森直接晋级淘汰赛，多特蒙德和拜仁将参加淘汰赛附加赛，斯图加特和莱比锡被直接淘汰。参加欧联杯的法兰克福直接晋级淘汰赛，霍芬海姆被
检索到的段落8 :#### 欧冠新赛制格外刺激，曼城将在附加赛对阵皇马或拜仁 
2025-01-30 11:30 发布于：广东省
史上第一次，欧冠正赛在北京时间1月30日凌晨4时同时开始了多达18场比赛，堪比足球春晚，
-------------------
重排序后结果：
混合检索结果：
检索到的段落1 :面对36支球队采取联赛总排名的欧冠全新赛制，有的传统强队有点懵圈，曼城、皇马、拜仁都表现不佳

In [8]:
# 方法二：cohere
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank

# 初始化Cohere重排序器
os.environ["COHERE_API_KEY"] = "api-key"

compressor = CohereRerank(model="rerank-english-v3.0",top_n=5)  # 重排序后保留前5个
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=ensemble_retriever  # 使用之前的混合检索器
)


compressed_docs = compression_retriever.get_relevant_documents(query)
print(f"使用cohere重排序的结果：")
for i,doc in enumerate(compressed_docs):
    print(f"检索到的段落{i+1} :{doc.page_content[:100]}")

C:\Users\ASUS\AppData\Local\Temp\ipykernel_55836\4057331377.py:8: LangChainDeprecationWarning: The class `CohereRerank` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import CohereRerank``.
  compressor = CohereRerank(model="rerank-english-v3.0",top_n=5)  # 重排序后保留前5个


使用cohere重排序的结果：
检索到的段落1 :#### 欧冠综合｜利物浦和巴萨直通16强
新华社 2025-01-30 11:12 北京
新华社柏林1月29日电（记者刘旸）本赛季欧冠联赛第一阶段29日结束全部8轮比赛，利物浦、巴萨、阿森纳等积分榜
检索到的段落2 :在欧冠淘汰赛附加赛中，常规赛排名最高的第9或第10位球队，将对阵排名最低的第23或24位球队；排名第11或第12位的球队，将对阵排名第21或第22位的球队；排名第13或第14位的球队，将对阵排名第19
检索到的段落3 :#### 五大联赛欧战积分排名 ，英超无愧第一联赛，意甲西甲相差无几
新欧洲三大杯联赛阶段已经全部结束，每个杯赛各有8支球队直接晋级下阶段淘汰赛，16支球队将参加淘汰赛附加赛争夺另外8个晋级名额，还有
检索到的段落4 :面对36支球队采取联赛总排名的欧冠全新赛制，有的传统强队有点懵圈，曼城、皇马、拜仁都表现不佳。最后一轮联赛开始前，竟然仅有利物浦和巴萨锁定16强入场券。9支种子队除了卫冕冠军皇马之外，其余球队按照欧战
检索到的段落5 :德甲：7支球队参加欧战，4支欧冠2支欧联杯1支欧协联。其中参加欧冠的勒沃库森直接晋级淘汰赛，多特蒙德和拜仁将参加淘汰赛附加赛，斯图加特和莱比锡被直接淘汰。参加欧联杯的法兰克福直接晋级淘汰赛，霍芬海姆被


#### 4. 创建问题改写和RAG模板(多种方法)

In [10]:
from langchain.prompts.chat import ChatPromptTemplate

# 通用回答prompt模板
def get_simple_answer_prompt():
    system_prompt = """\
    你是一个问答任务的助手，请依据以下检索出来的信息去回答问题，回答的字数控制在100字内：
    {context}
    """
    return ChatPromptTemplate([
        ("system", system_prompt),
        ("human", "{input}")  # 直接使用当前问题，不添加历史记录
    ])

##### 4.1. 仅向量检索

In [12]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

# 2. 创建向量索引器
vector_retriever = vector_store.as_retriever(search_kwargs={"k": 10})

# 3. 生成回答的 chain
vector_qa_prompt = get_simple_answer_prompt()
vector_qa_chain = create_stuff_documents_chain(llm, vector_qa_prompt)

# 4. 创建不使用历史的 RAG 链
vector_rag_chain = create_retrieval_chain(vector_retriever, vector_qa_chain)
vector_rag_chain.invoke({"input": "曼城附加赛的对手是谁"})


{'input': '曼城附加赛的对手是谁',
 'context': [Document(metadata={'_id': '16b8837ba70546808e6738d032853a62', '_collection_name': 'rag_collection'}, page_content='**曼城对决**\n在抽签仪式上，首先抽出第23、第24名球队，然后依次抽出剩余非种子球队。随后，首先抽出第15、16名球队，然后依次抽出种子球队。附加赛没有回避规则，球队可以对阵同一足协球队以及联赛阶段曾对阵过的球队。\n担任抽签嘉宾的蒂亚戈首先抽出布鲁日、葡萄牙体育，随后抽出曼城、凯尔特人。在结束了非种子球队的抽签之后，蒂亚戈抽出种子球队，他首先抽出巴黎圣日耳曼，大巴黎将对阵布雷斯特。\n在最为期待的对决中，蒂亚戈抽出了皇马，皇马将对阵曼城，这将是两队近5个赛季第4次交锋，而拜仁则对阵凯尔特人。\n2024-25赛季欧冠淘汰赛附加赛对阵：\n葡萄牙体育-多特蒙德\n布鲁日-亚特兰大\n凯尔特人-拜仁慕尼黑\n曼城-皇家马德里\n费耶诺德-AC米兰\n尤文图斯-埃因霍温\n摩纳哥-本菲卡\n布雷斯特-巴黎圣日耳曼\n附加赛首回合将在2月11日、12日进行，次回合则将在2月18日、19日进行。'),
  Document(metadata={'_id': '76af113624ae4814a27a4cba17e61651', '_collection_name': 'rag_collection'}, page_content='#### 欧冠附加赛抽签：曼城再战皇马，5年4次决战，拜仁大巴黎好签\n2025-01-31 19:29:28\u3000来源: 奥拜尔 \n北京时间1月31日19时，2024-25赛季欧冠淘汰赛附加赛抽签在瑞士尼翁欧足联总部进行。在附加赛里，曼城将对阵皇马，这将是欧冠决赛的翻版。\n**抽签规则**\n欧足联本赛季对欧冠、欧联杯赛制进行改革，联赛阶段前8名直接晋级16强，排名第9到第24位的球队通过两回合附加赛拿到另外8张门票。\n按照规则，排名第9到第16位的球队成为附加赛种子队，根据联赛阶段排名决定潜在对手，排名第9、第10的球队将对阵第23、24的球队，以此类推。\n同时，在八分之一决赛抽签中，联赛阶段排名第1、2位的球队，对手将是附

##### 4.2. 混合检索

In [ ]:
# 方法二：常规rag（不包含历史记录）+ 混合检索

# 2.1 向量检索
vector_retriever = vector_store.as_retriever(search_kwargs={"k": 10})  

# 2.2 关键词检索
bm25_retriever = BM25Retriever.from_texts(all_chunks, preprocess_func=lambda text:" ".join(jieba.cut(text)))
bm25_retriever.k =  10  # Retrieve top 10 results

# 2.3 混合检索
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, vector_retriever],
                                       weights=[0.4, 0.6])

# 3. 生成回答的 chain
hybrid_qa_prompt = get_simple_answer_prompt()
hybrid_qa_chain = create_stuff_documents_chain(llm, hybrid_qa_prompt)

# 4. 创建混合检索的 RAG 链
hybrid_rag_chain = create_retrieval_chain(ensemble_retriever, hybrid_qa_chain)

##### 4.3. 重排序

In [14]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
# 方法三：常规rag（不包含历史记录）+ 混合检索重排序

# 3.1 向量检索
vector_retriever = vector_store.as_retriever(search_kwargs={"k": 10})  

# 3.2 关键词检索
from langchain.retrievers import BM25Retriever, EnsembleRetriever
bm25_retriever = BM25Retriever.from_texts(all_chunks, preprocess_func=lambda text:" ".join(jieba.cut(text)))
bm25_retriever.k =  10  # Retrieve top 5 results

# 3.3 混合检索
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, vector_retriever],
                                       weights=[0.4, 0.6])

# 3.4 混合检索重排序
# 初始化Cohere重排序器
os.environ["COHERE_API_KEY"] = ""

compressor = CohereRerank(model="rerank-english-v3.0",top_n=10)  # 重排序后保留前10个
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=ensemble_retriever  # 使用之前的混合检索器
)

# 3. 生成回答的 chain
rrf_qa_prompt = get_simple_answer_prompt()
rrf_qa_chain = create_stuff_documents_chain(llm, rrf_qa_prompt)

# 4. 创建混合检索的 RAG 链
rrf_rag_chain = create_retrieval_chain(compression_retriever, rrf_qa_chain)

#### 5. RAG QA测试

In [15]:
# 构建测试数据集
import pandas as pd
file_path = "qa_dataset.xlsx"
df = pd.read_excel(file_path,sheet_name="positive")

questions = df["question"].astype(str).tolist()
ground_truths = df["ground_truth"].astype(str).tolist()

# 测试
# print(questions[:5])  # 预览前5个问题
# print(ground_truths[:5])  # 预览前5个答案


In [17]:
# 无历史记录结果记录：
from datasets import Dataset

answers = []
contexts = []

for query in questions:
    # 方法一：向量检索
    # answer = vector_rag_chain.invoke({"input": query})
    # answers.append(answer['answer'])
    # # 只存储相关文本，确保格式匹配
    # context = vector_retriever.get_relevant_documents(query)
    # contexts.append([doc.page_content for doc in context]) 

    # 方法二：混合检索
    # answer = hybrid_rag_chain.invoke({"input": query})
    # answers.append(answer['answer'])
    # context = ensemble_retriever.get_relevant_documents(query)
    # contexts.append([doc.page_content for doc in context]) 

    # 方法三：重排序
    answer = rrf_rag_chain.invoke({"input": query})
    answers.append(answer['answer'])
    context = compression_retriever.get_relevant_documents(query)
    contexts.append([doc.page_content for doc in context]) 

data = {
    "question": questions,
    "ground_truth": ground_truths,
    "answer": answers,
    "retrieved_contexts": contexts
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

c:\file\Anaconda3\envs\rag_test\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##### 使用ragas进行结果评估

In [18]:
os.environ["OPENAI_API_KEY"] = ""

from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    context_recall,
    context_precision,
    answer_relevancy,
    answer_correctness
)
from ragas.metrics._factual_correctness import FactualCorrectness

result = evaluate(
    dataset = dataset, 
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
        answer_correctness
    ],
)
 
df = result.to_pandas()
df


Evaluating: 100%|██████████| 300/300 [09:10<00:00,  1.83s/it]


,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy,answer_correctness
0,欧冠直通16强淘汰赛的球队有哪些？,[#### 欧冠综合｜利物浦和巴萨直通16强\n新华社 2025-01-30 11:12 北...,欧冠直通16强淘汰赛的球队包括利物浦、巴塞罗那、阿森纳、国际米兰、马德里竞技、勒沃库森、里尔...,利物浦、巴塞罗那、阿森纳、国际米兰、马德里竞技、勒沃库森、里尔、阿斯顿维拉,1.000000,1.000000,1.000000,0.841226,0.822926
1,曼城在附加赛的对手是谁？,[#### 欧冠新赛制格外刺激，曼城将在附加赛对阵皇马或拜仁 \n2025-01-30 11...,曼城在欧冠附加赛的对手是皇家马德里。根据抽签结果，曼城将与皇马进行两回合比赛，争夺16强的席...,曼城在欧冠附加赛的对手是皇家马德里。,0.900000,1.000000,1.000000,0.857099,0.541452
2,附加赛的赛制是什么？,[AC米兰客场1:2不敌萨格勒布迪纳摩，穆萨上半场吃到第二张黄牌，被罚下场。拜仁慕尼黑主场3...,欧冠附加赛将采取两回合制，首回合将于2月11日或12日进行，次回合将在2月18日或19日进行...,欧冠新赛制引入了淘汰赛附加赛，目的是在常规赛排名第9至第24名的16支球队中，决出最后8个晋...,1.000000,0.750000,1.000000,0.779169,0.687720
3,内马尔现在在哪只球队？,[#### 巴西球星内马尔重返桑托斯 承诺将竭尽所能助力队友\n2025年 02月 01日\...,内马尔目前重返了巴西球队桑托斯，这是他职业生涯开始的地方。他在2025年2月正式回归，并签署...,截至2025年2月1日，内马尔已与巴西的桑托斯俱乐部签约。,0.639286,1.000000,1.000000,0.805939,0.772564
4,他受了什么伤？,[一方面，球星受伤病影响和状态的下滑可能使巨额投资沦为“沉没成本”，俱乐部未来引援时肯定会更...,问题中没有明确指出具体的“他”指的是谁。根据提供的信息，有提到内马尔在利雅得新月期间因连续的...,内马尔在2023年代表巴西国家队比赛时遭遇左膝半月板撕裂和前交叉韧带断裂的重伤，导致长期缺阵...,0.000000,0.000000,0.666667,0.000000,0.347452
5,他为何与新月解约？,[一方面，球星受伤病影响和状态的下滑可能使巨额投资沦为“沉没成本”，俱乐部未来引援时肯定会更...,内马尔与利雅得新月解约的主要原因是其频繁受伤和低出勤率，导致实际贡献与高昂薪资严重失衡，给俱...,内马尔因伤病导致出勤率极低，仅代表球队出场7次，贡献1球3助攻，与其高昂的1亿欧元年薪严重不...,1.000000,1.000000,1.000000,0.753528,0.516342
6,有哪些传统强队今年进入了附加赛?,[#### 欧冠新赛制格外刺激，曼城将在附加赛对阵皇马或拜仁 \n2025-01-30 11...,今年进入欧冠附加赛的传统强队有皇马、拜仁、曼城、AC米兰、尤文图斯和巴黎圣日耳曼等。这些球队...,曼城、皇马、拜仁、AC米兰、尤文图斯、巴黎圣日耳曼,0.895685,1.000000,0.833333,0.853307,0.785211
7,Neymar在沙特联赛的表现如何？,[他说：“没有人想到20多轮比赛后我们能有这么多积分，我也没料到我们能以这么大的优势领先。在...,内马尔在沙特联赛的表现不尽如人意。他于2023年8月以9000万欧元转会费加盟利雅得新月，在...,由于长期伤病，内马尔在加盟后仅踢了7场比赛，贡献1球3助攻，竞技表现远未达到预期。,0.533333,1.000000,1.000000,0.848904,0.474851
8,新赛制对球员影响如何？,[**利益最大化的隐忧**\n足球比赛最核心的资源是球员，欧足联希望在欧冠实现利益最大化，球...,欧冠新赛制下，参赛球队从32支扩至36支，小组赛轮次从6轮增至8轮，导致赛程更加密集，球员身...,赛程更密集，球员伤病风险加大。例如，曼城的罗德里因伤赛季报销，皇马的 卡瓦哈尔和米利唐长期缺...,0.944444,1.000000,1.000000,0.823239,0.476471
9,罗德里现在在哪个队？,[罗德里打入全场唯一进球。\n“我之前拿了那么多冠军，是因为有梅西，现在则是有哈兰德。”赛前...,罗德里目前效力于曼城队。他在球队中担任中场，表现极为出色，不仅在2024年首次荣膺金球奖，还...,曼城足球俱乐部。,0.930556,1.000000,1.000000,0.863501,0.424162


#### 6. 结合历史记录提高多轮对话能力

In [16]:
from langchain.prompts import MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory
from langchain.chains.history_aware_retriever import create_history_aware_retriever

# 结合历史记录并改写问题

# 改写问题prompt
def get_contextualize_question_prompt():
    """
    基于历史记录来改写用户问的问题
    :return:
    """
    system_prompt = """\
    请根据聊天历史和最后用户的问题，改写用户最终提出的问题。
    你只需要改写用户最终的问题，请不要回答问题
    没有聊天历史则将用户问题直接返回，有聊天历史则进行改写
    """
    contextualize_question_prompt = ChatPromptTemplate([
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ])
    return contextualize_question_prompt

# 提问prompt
def get_answer_prompt():
    system_prompt = """\
    你是一个问答任务的助手，请依据以下检索出来的信息去回答问题，回答的字数控制在100字内：
    {context}
    """
    qa_prompt = ChatPromptTemplate([
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ])
    return qa_prompt

# 获取历史记录
def get_session_history(session_id:str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# 定义检索器
retriever = ensemble_retriever

# contextualize question
question_prompt = get_contextualize_question_prompt()
# 改写链：结合上下文改写用户问题
history_aware_retriever = create_history_aware_retriever(llm, retriever, question_prompt)

# qa chain
qa_prompt_template = get_answer_prompt()
# 问答链：根据问题和参考内容生成答案
qa_chain = create_stuff_documents_chain(llm, qa_prompt_template)
rag_chain = create_retrieval_chain(history_aware_retriever, qa_chain)

# with history
store = {}
# 在 rag_chain 中添加 chat_history
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
)

# 改写用户内容部分
contextualize_question_chain = RunnableWithMessageHistory(
    question_prompt | llm,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history"
    )

#### RAG系统测试

In [ ]:
# 结合历史记录的RAG提问，包含改写问题和回答两部分，回答end时结束会话

while(True):
    inputs = input("请输入：")
    print(f"❓输入的问题：{inputs}")
    if inputs == "end":
        break

    res = contextualize_question_chain.invoke({
        "input": inputs
    }, config={
        "configurable": {"session_id": "test456"}
    })
    print("🤖改写后内容：\n" + res)

    res = conversational_rag_chain.invoke({
        "input": inputs
    }, config={
        "configurable": {"session_id": "test123"}
    })
    print("📕回答：\n" + res["answer"])
    print("-----------------------------")


In [ ]:
# 测试常规RAG

questions = ["2025年欧冠直通16强淘汰赛的球队有哪些？",
             "第一个球队在哪个联赛？",
             "有哪些传统强队今年进入了附加赛?",
             "附加赛的赛制是什么？",
             "2023年的决赛是哪两只球队踢？"]

for query in questions:
    print(f"❓输入的问题：{query}")

    answer = vector_rag_chain.invoke({"input": query})
    print("📕回答：\n"+answer["answer"])

❓输入的问题：2025年欧冠直通16强淘汰赛的球队有哪些？
📕回答：
2025年欧冠直通16强的球队包括利物浦、巴塞罗那、阿森纳、国际米兰、马德里竞技、勒沃库森、里尔和阿斯顿维拉。这些球队在欧冠联赛第一阶段结束后，凭借积分榜前8名的位置直接获得了16强席位。
❓输入的问题：第一个球队在哪个联赛？
📕回答：
问题中提到的“第一个球队”不明确，但根据提供的信息，如果指的是欧战表现最突出的球队，那么英超的利物浦和巴萨（西甲）均表现优异，其中利物浦所在的联赛是英超。若需具体指向，请明确是哪支球队。
❓输入的问题：有哪些传统强队今年进入了附加赛?
📕回答：
根据欧冠新赛制，今年进入附加赛的传统强队有：

1. **曼城** - 常规赛排名第22位。
2. **皇家马德里** - 常规赛排名第11位。
3. **拜仁慕尼黑** - 常规赛排名第12位。
4. **AC米兰** - 常规赛排名第13位。
5. **尤文图斯** - 常规赛排名第19位。
6. **多特蒙德** - 常规赛排名第10位。

这些球队将通过两回合附加赛争夺16强席位。
❓输入的问题：附加赛的赛制是什么？
📕回答：
欧冠附加赛将采取两回合制，首回合在2月11日进行，次回合在2月18日进行。排名第9到第16位的球队成为种子队，对阵排名第17到第24位的非种子队。排名较高的球队原则上抽到实力较弱的球队，但没有回避规则，球队可以对阵同一足协球队或联赛阶段曾交手过的球队。
❓输入的问题：2023年的决赛是哪两只球队踢？
📕回答：
2023年欧冠决赛由曼城对阵国际米兰。这场比赛中，曼城以1-0战胜国米，夺得冠军，实现了队史第二次捧起欧冠奖杯的愿望。
